In [ ]:
%load_ext autoreload
%autoreload 2
import json
import mycode.metrics as metrics
import mycode.utils as u
import mycode.vap as vap

In [ ]:
# the goal is to:
# - print the feature output
# - Try taking some value and putting it under some variable
# - extract it to a separate searcher

In [ ]:
# Create a sample VAP
from vespa.package import (ApplicationPackage, ServicesConfiguration, Field, Schema, Document, RankProfile, Function)
from vespa.configuration.services import services, container, document_api, search, chain, searcher, \
    document_processing, content, documents, document, redundancy

doc_schema = Schema(
    name="doc",
    document=Document(
        fields=[
            Field(
                name="id",
                type="int",
                indexing=["attribute"],
                attribute=["fast-search"],
            ),
            Field(
                name="str",
                type="string",
                indexing=["attribute"],
            ),
            Field(
                name="my_tensor",
                type="tensor<float>(x[1])",
                indexing=["attribute"],
            )
        ]
    ),
    rank_profiles=[
        RankProfile(
            name='fields',
            inherits='unranked',
            first_phase="0",
            functions=[
                Function(
                    name='id',
                    expression='attribute(id)'
                ),
                Function(
                    name='str',
                    expression='tensorFromLabels(attribute(str))'
                ),
                Function(
                    name='my_tensor',
                    expression='attribute(my_tensor)'
                ),
            ],
            match_features=[
                'id',
                'str',
                'my_tensor',
            ],
        )
    ]
)

services_config = ServicesConfiguration(
    application_name="test",
    services_config=services(
        container(
            document_api(),
            document_processing(),
            search(
                chain(
                    searcher(
                        id="lt.jocas.examples.MatchFeaturesMapperSearcher",
                        bundle="match-features-mapper-bundle",
                    ),
                    id="match_features_mapper",
                    inherits='vespa'
                )
            ),
            id="container",
            version="1.0"
        ),
        content(
            redundancy(1),
            documents(
                document(type="doc", mode="index"),
            ),
            id="content",
            version="1.0"
        )
    )
)

application_package = ApplicationPackage(
    name='test',
    schema=[doc_schema],
    services_config=services_config,
)

In [ ]:
# Package into a bundle jar
!(cd ../examples/match-features-mapper-bundle && mvn -f pom.xml package)
# There should be a file `../examples/match-features-mapper-bundle/target/match-features-mapper-bundle-0.0.1-deploy.jar`
!ls -alh ../examples/match-features-mapper-bundle/target/match-features-mapper-bundle-0.0.1-deploy.jar

In [ ]:
from pathlib import Path

# Currently, pyvespa provides no way to add bundle jars nicely into VAP
# So, we need to do a little dance and add it manually.

tmp_vap_folder = '_tmp'
application_package.to_files(Path(tmp_vap_folder))

vap.add_bundles(
    application_root=tmp_vap_folder,
    bundles=['../examples/match-features-mapper-bundle/target/match-features-mapper-bundle-0.0.1-deploy.jar']
)

from vespa.deployment import VespaDocker

# In case running colima on macos run the following
# !sudo ln -sf $HOME/.colima/default/docker.sock /var/run/docker.sock
vespa_docker = VespaDocker(
    container_image="vespaengine/vespa:8.588.8",
)

client = vespa_docker.deploy_from_disk(
    application_name='test',
    application_root=Path(tmp_vap_folder),
)

In [ ]:
def compile_deploy():
    !(cd ../examples/match-features-mapper-bundle && mvn -f pom.xml package)
    tmp_vap_folder = '_tmp'
    application_package.to_files(Path(tmp_vap_folder))
    vap.add_bundles(
        application_root=tmp_vap_folder,
        bundles=['../examples/match-features-mapper-bundle/target/match-features-mapper-bundle-0.0.1-deploy.jar']
    )
    vap.redeploy_from_disk(
        docker=vespa_docker,
        application_root=tmp_vap_folder
    )

In [ ]:
compile_deploy()

In [ ]:
# Create and feed 1 dummy doc
docs = [
    {
        'id': f'{1}',
        'fields': {
            'id': 1,
            'str': 'foo',
            'my_tensor': [2]
        }
    }
]

client.feed_iterable(docs, schema="doc", namespace="doc", callback=vap.feed_callback)

In [ ]:
compile_deploy()
# print(client.query(body={
#     'yql': 'select matchfeatures from sources * where true',
#     'searchChain': 'match_features_mapper',
#     'ranking': 'fields',
#     'presentation.summary': 'default',
#     'trace.level': 1,
#     'presentation.format.tensors': 'short-value',
# }).json)

In [ ]:
print(client.query(body={
    'yql': 'select matchfeatures from sources doc where true',
    'searchChain': 'match_features_mapper',
    'ranking': 'fields',
    'presentation.summary': 'default',
    # 'trace.level': 1,
    'presentation.format.tensors': 'short-value',
}).json)